# Model Training

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.callbacks import ReduceLROnPlateau



In [45]:
df = pd.read_csv("/Users/khushpatel/Desktop/ML Projects/Emotion Detection/CREMA-D MODEL/CREMA_audio_features.csv")

In [87]:

encoder = OneHotEncoder()
encoder.fit(df[['label']])
encoded_labels = pd.DataFrame(encoder.transform(df[['label']]).toarray(), columns = encoder.get_feature_names_out(['label']))
print(encoded_labels)
df_encoded = pd.concat([df, encoded_labels], axis=1)

# Drop the original 'label' column
df_encoded.drop(columns=['label'], inplace=True)

df_encoded.columns

      label_ANG  label_DIS  label_FEA  label_HAP  label_NEU  label_SAD
0           1.0        0.0        0.0        0.0        0.0        0.0
1           1.0        0.0        0.0        0.0        0.0        0.0
2           0.0        0.0        0.0        0.0        1.0        0.0
3           0.0        0.0        0.0        0.0        1.0        0.0
4           0.0        1.0        0.0        0.0        0.0        0.0
...         ...        ...        ...        ...        ...        ...
7437        1.0        0.0        0.0        0.0        0.0        0.0
7438        1.0        0.0        0.0        0.0        0.0        0.0
7439        1.0        0.0        0.0        0.0        0.0        0.0
7440        1.0        0.0        0.0        0.0        0.0        0.0
7441        0.0        0.0        0.0        0.0        1.0        0.0

[7442 rows x 6 columns]


Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '162', '163', '164', '165', 'label_ANG', 'label_DIS', 'label_FEA',
       'label_HAP', 'label_NEU', 'label_SAD'],
      dtype='object', length=172)

In [90]:
target = ['label_ANG', 'label_DIS', 'label_FEA',
       'label_HAP', 'label_NEU', 'label_SAD']

X = df_encoded.drop(columns=target).values
Y = df_encoded[target].values

scaled = StandardScaler()
X_scaled = scaled.fit_transform(X)

X_train,X_test,y_train,y_test =  train_test_split(X,Y,test_size=0.2,random_state=42)
print(Y.shape)

(7442, 6)


In [97]:




X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)


# Define the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1),
                ))  # L2 regularization with lambda=0.01
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', ))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', ))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output from convolutional layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(256, activation='relu'),)
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'),)
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))  # Output layer with softmax activation for multi-class classification

optimier = Adam(learning_rate=0.001)

model.compile(optimizer=optimier,loss='categorical_crossentropy',metrics=['accuracy'])


# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/25
94/94 [==============================] - 4s 38ms/step - loss: 1.7107 - accuracy: 0.2871 - val_loss: 1.5491 - val_accuracy: 0.3371
Epoch 2/25
94/94 [==============================] - 4s 41ms/step - loss: 1.5328 - accuracy: 0.3511 - val_loss: 1.4505 - val_accuracy: 0.4009
Epoch 3/25
94/94 [==============================] - 4s 43ms/step - loss: 1.4682 - accuracy: 0.3865 - val_loss: 1.4690 - val_accuracy: 0.4043
Epoch 4/25
94/94 [==============================] - 4s 41ms/step - loss: 1.4610 - accuracy: 0.3922 - val_loss: 1.3977 - val_accuracy: 0.4238
Epoch 5/25
94/94 [==============================] - 4s 42ms/step - loss: 1.4282 - accuracy: 0.4045 - val_loss: 1.4116 - val_accuracy: 0.4372
Epoch 6/25
94/94 [==============================] - 4s 40ms/step - loss: 1.4019 - accuracy: 0.4186 - val_loss: 1.3749 - val_accuracy: 0.4406
Epoch 7/25
94/94 [==============================] - 4s 41ms/step - loss: 1.3801 - accuracy: 0.4302 - val_loss: 1.4082 - val_accuracy: 0.4184
Epoch 8/25
94

In [99]:
model.save('emotion_recognition_model(2).h5')

## Traning the same model on a dataset having different extracted features 


In [100]:
df = pd.read_csv("/Users/khushpatel/Desktop/ML Projects/Emotion Detection/CREMA-D MODEL/new_features.csv")

In [101]:

encoder = OneHotEncoder()
encoder.fit(df[['label']])
encoded_labels = pd.DataFrame(encoder.transform(df[['label']]).toarray(), columns = encoder.get_feature_names_out(['label']))
print(encoded_labels)
df_encoded = pd.concat([df, encoded_labels], axis=1)

# Drop the original 'label' column
df_encoded.drop(columns=['label'], inplace=True)

df_encoded.columns

      label_ANG  label_DIS  label_FEA  label_HAP  label_NEU  label_SAD
0           1.0        0.0        0.0        0.0        0.0        0.0
1           1.0        0.0        0.0        0.0        0.0        0.0
2           0.0        0.0        0.0        0.0        1.0        0.0
3           0.0        0.0        0.0        0.0        1.0        0.0
4           0.0        1.0        0.0        0.0        0.0        0.0
...         ...        ...        ...        ...        ...        ...
7437        1.0        0.0        0.0        0.0        0.0        0.0
7438        1.0        0.0        0.0        0.0        0.0        0.0
7439        1.0        0.0        0.0        0.0        0.0        0.0
7440        1.0        0.0        0.0        0.0        0.0        0.0
7441        0.0        0.0        0.0        0.0        1.0        0.0

[7442 rows x 6 columns]


Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '151', '152', '153', '154', 'label_ANG', 'label_DIS', 'label_FEA',
       'label_HAP', 'label_NEU', 'label_SAD'],
      dtype='object', length=161)

In [102]:
target = ['label_ANG', 'label_DIS', 'label_FEA',
       'label_HAP', 'label_NEU', 'label_SAD']

X = df_encoded.drop(columns=target).values
Y = df_encoded[target].values

scaled = StandardScaler()
X_scaled = scaled.fit_transform(X)

X_train,X_test,y_train,y_test =  train_test_split(X,Y,test_size=0.2,random_state=42)
print(Y.shape)

(7442, 6)


In [104]:
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)


# Define the CNN model
model = Sequential()

# Add convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1),
                ))  # L2 regularization with lambda=0.01
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', ))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', ))
model.add(MaxPooling1D(pool_size=2))

# Flatten the output from convolutional layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(256, activation='relu'),)
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'),)
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))  # Output layer with softmax activation for multi-class classification

optimier = Adam(learning_rate=0.001)

model.compile(optimizer=optimier,loss='categorical_crossentropy',metrics=['accuracy'])


# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Epoch 1/100
94/94 [==============================] - 5s 47ms/step - loss: 2.0839 - accuracy: 0.2474 - val_loss: 1.6709 - val_accuracy: 0.2754
Epoch 2/100
94/94 [==============================] - 4s 43ms/step - loss: 1.6374 - accuracy: 0.2873 - val_loss: 1.5374 - val_accuracy: 0.3479
Epoch 3/100
94/94 [==============================] - 7s 79ms/step - loss: 1.5504 - accuracy: 0.3350 - val_loss: 1.5041 - val_accuracy: 0.3653
Epoch 4/100
94/94 [==============================] - 5s 49ms/step - loss: 1.5287 - accuracy: 0.3454 - val_loss: 1.5303 - val_accuracy: 0.3479
Epoch 5/100
94/94 [==============================] - 4s 41ms/step - loss: 1.5068 - accuracy: 0.3728 - val_loss: 1.4515 - val_accuracy: 0.3996
Epoch 6/100
94/94 [==============================] - 4s 43ms/step - loss: 1.4687 - accuracy: 0.3852 - val_loss: 1.4511 - val_accuracy: 0.3996
Epoch 7/100
94/94 [==============================] - 6s 60ms/step - loss: 1.4486 - accuracy: 0.3906 - val_loss: 1.4327 - val_accuracy: 0.4150
Epoch 

KeyboardInterrupt: 

In [108]:
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_test = X_test.reshape(-1, X_test.shape[1], 1)

model=Sequential()
model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
model.add(Dropout(0.2))

model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=6, activation='softmax'))
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])


history=model.fit(X_train, y_train, batch_size=64, epochs=100, validation_data=(X_test, y_test), callbacks=[reduce_lr])

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/100
94/94 [==============================] - 14s 149ms/step - loss: 2.3788 - accuracy: 0.1666 - val_loss: 1.8100 - val_accuracy: 0.1713 - lr: 0.0010
Epoch 2/100
94/94 [==============================] - 15s 160ms/step - loss: 1.7921 - accuracy: 0.1641 - val_loss: 1.7910 - val_accuracy: 0.1692 - lr: 0.0010
Epoch 3/100
94/94 [==============================] - 18s 188ms/step - loss: 1.7915 - accuracy: 0.1685 - val_loss: 1.7858 - val_accuracy: 0.1692 - lr: 0.0010
Epoch 4/100
94/94 [==============================] - 16s 166ms/step - loss: 1.7912 - accuracy: 0.1713 - val_loss: 1.7909 - val_accuracy: 0.1692 - lr: 0.0010
Epoch 5/100
94/94 [==============================] - 16s 174ms/step - loss: 1.7913 - accuracy: 0.1697 - val_loss: 1.7907 - val_accuracy: 0.1666 - lr: 0.0010
Epoch 6/100
94/94 [==============================] - 15s 162ms/step - loss: 1.7884 - accuracy: 0.1735 - val_loss: 1.7857 - val_accuracy: 0.2310 - lr: 0.0010
Epoch 7/100
94/94 [==============================] - 15s 1

In [109]:
model.save('new_model.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
